In [1]:
cd ..

/home/is/akiyoshi-n/my-project


In [3]:
import os
# 使用するGPUを指定. この環境変数の場所は，pytorchをimportする前に入れる
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from pathlib import Path
from datetime import datetime
from src.my_project.dataset import load_dataset_2class_classification,load_dataset_4class_Multi_classification, split_multilabel_data, split_data_by_index, load_dataset_3class_Multi_classification, split_same_texts_data
from src.my_project.train_v2 import ActivityPrediction
from sklearn.model_selection import train_test_split
from src.my_project.dataset import load_multiclass_dataset
import wandb
import numpy as np

In [4]:
DATASET_PATH = Path('/home/is/akiyoshi-n/my-project/data')
# 本日の日付
timestamp = datetime.now().strftime("%Y-%m-%d")
# 出力先ディレクトリ
output_dir = Path('/home/is/akiyoshi-n/my-project/outputs/{}'.format(timestamp))

In [5]:
# 最大トークン数
MAX_LEN = 128
# バッチサイズ
BATCH_SIZE = 16
# エポック数
NUM_EPOCHS = 100
# 学習率
LEARNING_RATE = 2e-5
# Cross Validation時のFold数
NUM_FOLDS = 3
# 早期停止のための忍耐値
PATIENCE = 10
# 乱数シード
SEED = 2023
# クラス数
NUM_LABELS = 21
# 閾値
THRESH = 0.5

### unursoデータ

In [6]:
import re
import pandas as pd

# 正規表現パターンと対応する置換表現を辞書で定義
replacements = {
    r"@(\S*)": "[USR]", # ユーザ名を[USR]に置換
    r"http(\S*)": "[URL]", # URLを[URL]に置換
}

# 文中のマッチした箇所を対応する置換表現に置換する関数
def apply_regex(text):
    for pattern, replacement in replacements.items():
        text = re.sub(pattern, replacement, text)
    return text

# テキストファイルを読み込み、1つ目と3つ目のフィールドに前処理を適用し、それらを出力する関数
def preprocess_file(input_file):
    with open(input_file, 'r', encoding='utf-8') as file:
        processed_data = []
        for line in file:
            fields = line.strip().split('\t')
            # 2つ目のフィールド（前処理なし）と3つ目のフィールド（前処理あり）4つ目のフィールド（前処理なし）を取得
            first_field = fields[1] # ユーザ名
            third_field = apply_regex(fields[2]) # ツイート本文
            forth_field = fields[3] # ツイート日時
            processed_data.append([first_field, third_field, forth_field])
    return processed_data

In [7]:
# 関数を使用してファイルを前処理し、結果をDataFrameに格納
input_file = f'{DATASET_PATH}/unurso_users85.txt' # 入力ファイル名
processed_data = preprocess_file(input_file)
column_names = ['Name','text','time']
# DataFrameを作成
df = pd.DataFrame(processed_data, columns=column_names)

In [9]:
import pandas as pd
from transformers import AutoTokenizer
from src.my_project.dataset import preprocess_for_Trainer
import numpy as np
# 東北大BERT-v3
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v3'
# tokenizerの定義
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# テキストデータを取得
texts = df['text'].values.tolist()
# テキストを含む辞書型のデータセットとして返す
test_dataset = {
    'texts': texts
}

In [10]:
single_raw_data = load_dataset_2class_classification(f"{DATASET_PATH}/act_classification_final_ChatGPT4.xlsx")

In [11]:
# データの読み込み
multi_data, class_name = load_multiclass_dataset(f"{DATASET_PATH}/act_classification_final_ChatGPT4.xlsx")

In [12]:
# testデータと訓練に使用するデータに分割
multi_data, multi_eval_data = split_multilabel_data(data=multi_data, test_size=0.2, SEED=SEED)

In [13]:
dataset_labels_np = np.array(multi_data['labels'])
test_data_labels_np = np.array(multi_eval_data['labels'])
print(dataset_labels_np.sum(axis=0))
print(test_data_labels_np.sum(axis=0))

[ 23  17  22  38   6  51  22   9  15   6   9 110  22  16  66  10  43  10
   9   5 489]
[  6   4   6   9   2  13   6   2   4   2   2  28   6   4  17   3  11   3
   2   1 122]


In [14]:
single_data = split_same_texts_data(multi_data, single_raw_data)
single_eval_data = split_same_texts_data(multi_eval_data, single_raw_data)

In [15]:
# 東北大BERT-v3
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v3'
Classifier_model = ActivityPrediction(model_name = MODEL_NAME, seed=SEED, num_labels=21)

In [16]:
prediction = Classifier_model.train_and_predict(single_data, single_eval_data, multi_data, multi_eval_data, test_dataset, test_dataset, MAX_LEN, NUM_EPOCHS, LEARNING_RATE, BATCH_SIZE, PATIENCE, output_dir, project_name='2step_train_and_predict')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Parameter 'fn_kwargs'={'tokenizer': BertJapaneseTokenizer(name_or_path='cl-tohoku/bert-base-japanese-v3', vocab_size=32768, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False,

Map:   0%|          | 0/954 [00:00<?, ? examples/s]

Map:   0%|          | 0/246 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.715100,0.654689,0.593496,0.592175
2,0.610400,0.612551,0.678862,0.678218
3,0.507800,0.580849,0.719512,0.717829
4,0.376100,0.609525,0.695122,0.692884
5,0.217900,0.780483,0.699187,0.698869
6,0.091700,1.030634,0.707317,0.705742
7,0.029200,1.253759,0.752033,0.751996
8,0.010200,1.487032,0.735772,0.735768
9,0.005800,1.656269,0.723577,0.722679
10,0.017400,1.761637,0.707317,0.705370


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Map:   0%|          | 0/124 [00:00<?, ? examples/s]

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1,Class F1,Class Recall,Class Precision
1,0.631200,0.547102,0.000000,0.048150,"[0.0, 0.0, 0.0, 0.147, 0.0, 0.128, 0.0, 0.0, 0.022, 0.0, 0.017, 0.333, 0.167, 0.0, 0.111, 0.038, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.556, 0.0, 0.231, 0.0, 0.0, 0.25, 0.0, 0.5, 0.429, 0.167, 0.0, 0.059, 0.667, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.085, 0.0, 0.088, 0.0, 0.0, 0.011, 0.0, 0.009, 0.273, 0.167, 0.0, 1.0, 0.02, 0.0, 0.0, 0.0, 0.0]"
2,0.429700,0.297585,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,0.242800,0.184984,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,0.172600,0.146454,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
5,0.145100,0.128977,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
6,0.131100,0.120426,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
7,0.127000,0.115593,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
8,0.123900,0.112781,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
9,0.121100,0.111256,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
10,0.117500,0.109267,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


Map:   0%|          | 0/150584 [00:00<?, ? examples/s]

Map:   0%|          | 0/28195 [00:00<?, ? examples/s]

In [19]:
# predictionをpickleで保存
import pickle
with open(f'{DATASET_PATH}/unurso_2step_21class_pred.pkl', 'wb') as f:
    pickle.dump(prediction, f)

In [20]:
# predictionを読み込む
import pickle
with open(f'{DATASET_PATH}/unurso_2step_21class_pred.pkl', 'rb') as f:
    a = pickle.load(f)

In [36]:
# 列の合計が0の個数
print(np.array(prediction).sum(axis=1).tolist().count(0))
print(len(prediction))

18992
150584


In [37]:
new_df = pd.DataFrame(prediction)
new_df

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150579,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
150580,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
150581,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
150582,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [38]:
action_df = pd.concat([df, new_df], axis=1)

In [47]:
action_df = action_df[['Name',0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]

In [43]:
# 0~20の列の合計が0の行を削除
action_df = action_df.drop(action_df.index[np.array(prediction).sum(axis=1)==0])

In [45]:
action_df.shape

(131592, 24)

In [51]:
action_df

,Name,0,1,2,3,4,5,6,7,8,...,11,12,13,14,15,16,17,18,19,20
1,3zhen,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,3zhen,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,3zhen,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,3zhen,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
6,3zhen,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150578,zubora_sweet,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
150579,zubora_sweet,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
150581,zubora_sweet,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
150582,zubora_sweet,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [57]:
# unursoの分析
# ユーザ名でグループ化し，平均を計算
action_df.groupby('Name').mean().mean(axis=0)

0     0.019909
1     0.000091
2     0.000569
3     0.016643
4     0.000008
5     0.021303
6     0.009267
7     0.000008
8     0.002007
9     0.000008
10    0.000008
11    0.035134
12    0.007764
13    0.000008
14    0.023974
15    0.000008
16    0.005291
17    0.000036
18    0.000008
19    0.000008
20    0.858123
dtype: float64

In [69]:
action_df.groupby('Name').mean().var(axis=0)

0     4.300997e-04
1     1.358307e-07
2     2.547929e-06
3     4.932580e-04
4     5.925965e-09
5     5.217930e-04
6     1.855748e-04
7     5.925965e-09
8     1.982317e-05
9     5.925965e-09
10    5.925965e-09
11    1.615300e-03
12    8.933884e-05
13    5.925965e-09
14    1.586234e-03
15    5.925965e-09
16    1.412615e-04
17    7.278345e-08
18    5.925965e-09
19    5.925965e-09
20    6.116105e-03
dtype: float64

: 

### unurso_4class

In [67]:
# pickleでデータを読み込む
import pickle
with open(f'{DATASET_PATH}/urso_pred_4class.pkl', 'rb') as f:
    a2 = pickle.load(f)

In [68]:
a2

array([[0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]], dtype=float32)